In [98]:
import os
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
st = stopwords.words('english')

In [99]:
train = pd.read_csv('C:\\Users\\shreyas.raghavan\\Desktop\\NLP\\ASSN-W4\\Assignment sarcasm train data.csv')
test = pd.read_csv('C:\\Users\\shreyas.raghavan\\Desktop\\NLP\\ASSN-W4\\Assignment sarcasm test data.csv')

In [100]:
train['label'] =np.where(train['label']=='sarcastic',1,0)

In [101]:
train.head()

,ID,tweet,label
0,T000452360,@alleygirl2409 until i\'m and all the old men ...,1
1,T000452361,"b""@sarinas it had been chanted peacefully you ...",1
2,T000452365,b'Wow I really forgot how much I love the traf...,1
3,T000452367,b'Love having no voice $$SAR$$',1
4,T000452368,b'Apparently i have to sacrifice a goat and do...,1


In [102]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
comb = train.append(test, ignore_index=True)

In [103]:
import re
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt   

In [104]:
# pre-processing
comb['clean'] = np.vectorize(remove_pattern)(comb['tweet'], "@[\w]*")
comb['clean'] = comb['clean'].str.replace("[^a-zA-Z#]", " ")
comb['clean'] = comb['clean'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))

In [105]:
tokenized_tweet = comb['clean'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [until, and, all, the, old, men, will, finally...
1    [it, had, been, chanted, peacefully, you, can,...
2    [Wow, really, forgot, how, much, love, the, tr...
3                       [Love, having, no, voice, SAR]
4    [Apparently, have, to, sacrifice, goat, and, d...
Name: clean, dtype: object

In [106]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [until, and, all, the, old, men, will, final, ...
1    [it, had, been, chant, peac, you, can, deni, #...
2    [wow, realli, forgot, how, much, love, the, tr...
3                          [love, have, no, voic, sar]
4    [appar, have, to, sacrific, goat, and, do, rai...
Name: clean, dtype: object

In [107]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
comb['clean'] = tokenized_tweet

In [108]:
comb.head()

,ID,label,tweet,clean
0,T000452360,1.0,@alleygirl2409 until i\'m and all the old men ...,until and all the old men will final date me #...
1,T000452361,1.0,"b""@sarinas it had been chanted peacefully you ...",it had been chant peac you can deni #hypocrisy...
2,T000452365,1.0,b'Wow I really forgot how much I love the traf...,wow realli forgot how much love the traffic scene
3,T000452367,1.0,b'Love having no voice $$SAR$$',love have no voic sar
4,T000452368,1.0,b'Apparently i have to sacrifice a goat and do...,appar have to sacrific goat and do rain danc t...


In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000)
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(comb['clean'])

# ##VALIDATING THE DATA SET

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_tfidf = tfidf[:45529,:]
test_tfidf = tfidf[45530:,:]

# splitting data into training and validation set
xtrain_tfidf, xvalid_tfidf, ytrain, yvalid = train_test_split(train_tfidf, train['label'], random_state=42, test_size=0.3)


In [111]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
# Gini-model


from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
#X, y = make_hastie_10_2(random_state=0)
#X_train, X_test = X[:2000], X[2000:]
#y_train, y_test = y[:2000], y[2000:]


clf= GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(xtrain_tfidf, ytrain)
#from sklearn.ensemble import AdaBoostClassifier
#bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
 #                        algorithm="SAMME",
  #                       n_estimators=200)

#bdt.fit(xtrain_tfidf, ytrain)

In [113]:
gb_pred=predict=clf.predict_proba(xvalid_tfidf)


In [114]:
prediction_int = gb_pred[:,1] >= 0.5000000
prediction_int = prediction_int.astype(np.int)


In [115]:
f1_score(yvalid, prediction_int) 

0.8411895360315893

# Scoring the Test Data

In [116]:
from sklearn.metrics import f1_score

train_tfidf = tfidf[:45529,:]
test_tfidf = tfidf[45530:,:]

xtrain_tfidf = tfidf[:45529,:]
xvalid_tfidf = tfidf[45529:,:]
ytrain= train['label']

In [117]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
# Gini-model


from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
#X, y = make_hastie_10_2(random_state=0)
#X_train, X_test = X[:2000], X[2000:]
#y_train, y_test = y[:2000], y[2000:]


clf= GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(xtrain_tfidf, ytrain)
#from sklearn.ensemble import AdaBoostClassifier
#bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         #algorithm="SAMME",
                         #n_estimators=200)

#bdt.fit(xtrain_tfidf, ytrain)

In [118]:
gb_pred=clf.predict_proba(xvalid_tfidf)


In [119]:
prediction_int = gb_pred[:,1] >= 0.5000000
prediction_int = prediction_int.astype(np.int)
len(prediction_int)

45769

In [121]:
test.shape

(45769, 2)

In [122]:
test['label']=prediction_int

In [123]:
test['label'] =np.where(test['label']==1,'sarcastic','non-sarcastic')
test.head()

,ID,tweet,label
0,T000452358,b'oh yea that makes sense ',sarcastic
1,T000452359,Estas enfermedad a un cargo poltico tu como pb...,non-sarcastic
2,T000452362,"b""there's nothing like being on vacation and h...",sarcastic
3,T000452363,People who are sarcastic tend to be better pro...,sarcastic
4,T000452364,b'May I block you too RT But what if he or she...,non-sarcastic


In [125]:
test.to_csv('C:\\Users\\shreyas.raghavan\\Desktop\\NLP\\ASSN-W4\\Assignment_sarcasm_scored_data_shreyas_0487.csv')